<a href="https://colab.research.google.com/github/yepez26/BRFUNK_ismir_2024/blob/main/train_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

load data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
path = '/content/drive/MyDrive/UFRGS/mestrado/bert_class_data_aug/data'

In [ ]:
# the paper's topics names have a different name/numeration for convinence
columns = ['trecho', 0, 1, 2, 3, 4, 5, 6, 7, 9, 11, 12]

In [ ]:
data_og = pd.read_excel(path+'/data_annote_fab.xlsx')

data_og = data_og[columns]  # anotações originais

In [ ]:
data_gpt = pd.read_csv(path+'/gpt_aug_data.csv') # anotações geradas por gpt
data_gpt = data_gpt.rename(columns={'0':0,'1':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'9':9,'11':11,'12':12})
data_gpt = data_gpt[(data_gpt[6]==0)|(data_gpt['trecho_old'].isin(data_og[data_og[6]==1]['trecho'].unique())).reset_index(drop=True)]
data_gpt = data_gpt[columns]


In [ ]:
data_manual = pd.read_excel(path+'/data_augmentation_manual.xlsx')[columns]

In [ ]:
data_manual_359 = pd.read_excel(path+'/data_augmentation_359.xlsx')[columns]
data_manual_2 = pd.read_excel(path+'/data_augmentation_manual_2.xlsx')[columns]

In [ ]:
# concat annotated data from differents fonts
data = pd.concat([data_og,data_gpt,data_manual,data_manual_359,data_manual_2])

In [ ]:
data[[0, 1, 2, 3, 4, 5, 6, 7, 9, 11, 12]].sum()

0      68.0
1     139.0
2      77.0
3     236.0
4     179.0
5     122.0
6      84.0
7      90.0
9     231.0
11     96.0
12     65.0
dtype: float64

train

In [ ]:
!pip install transformers

In [ ]:

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
import torch.nn as nn

from tabulate import tabulate
from tqdm import trange
import random
import pickle

import ast

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
#load model
from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads


tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 150,
                       pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )

In [ ]:
def print_rand_sentence_encoding():
  '''Displays tokens, token IDs and attention mask of a random text sample'''
  index = random.randint(0, len(text) - 1)
  tokens = tokenizer.tokenize(tokenizer.decode(token_id[index]))
  token_ids = [i.numpy() for i in token_id[index]]
  attention = [i.numpy() for i in attention_masks[index]]

  table = np.array([tokens, token_ids, attention]).T
  print(tabulate(table,
                 headers = ['Tokens', 'Token IDs', 'Attention Mask'],
                 tablefmt = 'fancy_grid'))

In [ ]:

def b_tp(preds, labels):
  '''Returns True Positives (TP): count of correct predictions of actual class 1'''
  return sum([preds == labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_fp(preds, labels):
  '''Returns False Positives (FP): count of wrong predictions of actual class 1'''
  return sum([preds != labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_tn(preds, labels):
  '''Returns True Negatives (TN): count of correct predictions of actual class 0'''
  return sum([preds == labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_fn(preds, labels):
  '''Returns False Negatives (FN): count of wrong predictions of actual class 0'''
  return sum([preds != labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_metrics(preds, labels):
  '''
  Returns the following metrics:
    - accuracy    = (TP + TN) / N
    - precision   = TP / (TP + FP)
    - recall      = TP / (TP + FN)
    - specificity = TN / (TN + FP)
  '''
  preds = np.argmax(preds, axis = 1).flatten()
  labels = labels.flatten()
  tp = b_tp(preds, labels)
  tn = b_tn(preds, labels)
  fp = b_fp(preds, labels)
  fn = b_fn(preds, labels)
  b_accuracy = (tp + tn) / len(labels)
  b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
  b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
  b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 'nan'
  return b_accuracy, b_precision, b_recall, b_specificity

In [ ]:
split_test_size = 0.35
epochs = 40
batch_size = 32

for topic in [1,7]:

  df = data[data[topic]==1].reset_index(drop=True)
  aux = data[data[topic]==0].reset_index(drop=True)

  try:
    aux = aux.sample(len(df))
  except:
    df = df.sample(len(aux))

  df = pd.concat([df,aux])[['trecho',topic]]

  num_labels = 2

  text = df.trecho.values
  labels = df[topic].values


  ############

  token_id = []
  attention_masks = []


  for sample in text:
    encoding_dict = preprocessing(sample, tokenizer)
    token_id.append(encoding_dict['input_ids'])
    attention_masks.append(encoding_dict['attention_mask'])


  token_id = torch.cat(token_id, dim = 0)
  attention_masks = torch.cat(attention_masks, dim = 0)
  labels = torch.tensor(labels)

  ##########

  # print_rand_sentence_encoding()


  # Load the BertForSequenceClassification model
  model = BertForSequenceClassification.from_pretrained(
      'neuralmind/bert-base-portuguese-cased',
      num_labels = num_labels,
      output_attentions = False,
      output_hidden_states = False,
  )

  # model.resize_token_embeddings(len(tokenizer))
  # tokenizer.add_special_tokens({'pad_token': '<pad>'})


  # Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5. See: https://arxiv.org/pdf/1810.04805.pdf
  optimizer = torch.optim.AdamW(model.parameters(),
                                lr = 5e-5,
                                eps = 1e-08
                                )

  model.cuda()

  val_ratio = split_test_size
  # Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
  batch_size = 32

  # Indices of the train and validation splits stratified by labels
  train_idx, val_idx = train_test_split(
      np.arange(len(labels)),
      test_size = val_ratio,
      shuffle = True,
      stratify = labels,
      random_state=26)

  # Train and validation sets
  train_set = TensorDataset(token_id[train_idx],
                            attention_masks[train_idx],
                            labels[train_idx])

  val_set = TensorDataset(token_id[val_idx],
                          attention_masks[val_idx],
                          labels[val_idx])

  # Prepare DataLoader
  train_dataloader = DataLoader(
              train_set,
              sampler = RandomSampler(train_set),
              batch_size = batch_size
          )

  validation_dataloader = DataLoader(
              val_set,
              sampler = SequentialSampler(val_set),
              batch_size = batch_size
          )


  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  # Recommended number of epochs: 2, 3, 4. See: https://arxiv.org/pdf/1810.04805.pdf
  # epochs = 100

  epoch_val_metrics = []

  for _ in trange(epochs, desc = 'Epoch'):

      # ========== Training ==========

      # Set model to training mode
      model.train()

      # Tracking variables
      tr_loss = 0
      nb_tr_examples, nb_tr_steps = 0, 0

      for step, batch in enumerate(train_dataloader):
          batch = tuple(t.to(device) for t in batch)
          b_input_ids, b_input_mask, b_labels = batch
          optimizer.zero_grad()
          # Forward pass
          train_output = model(b_input_ids,
                              token_type_ids = None,
                              attention_mask = b_input_mask,
                              labels = b_labels)
          # Backward pass
          train_output.loss.backward()
          optimizer.step()
          # Update tracking variables
          tr_loss += train_output.loss.item()
          nb_tr_examples += b_input_ids.size(0)
          nb_tr_steps += 1

      # ========== Validation ==========

      # Set model to evaluation mode
      model.eval()

      # Tracking variables
      val_accuracy = []
      val_precision = []
      val_recall = []
      val_specificity = []

      for batch in validation_dataloader:
          batch = tuple(t.to(device) for t in batch)
          b_input_ids, b_input_mask, b_labels = batch
          with torch.no_grad():
            # Forward pass
            eval_output = model(b_input_ids,
                                token_type_ids = None,
                                attention_mask = b_input_mask)
          logits = eval_output.logits.detach().cpu().numpy()
          label_ids = b_labels.to('cpu').numpy()
          # Calculate validation metrics
          b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits, label_ids)
          val_accuracy.append(b_accuracy)
          # Update precision only when (tp + fp) !=0; ignore nan
          if b_precision != 'nan': val_precision.append(b_precision)
          # Update recall only when (tp + fn) !=0; ignore nan
          if b_recall != 'nan': val_recall.append(b_recall)
          # Update specificity only when (tn + fp) !=0; ignore nan
          if b_specificity != 'nan': val_specificity.append(b_specificity)



      print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
      print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
      print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
      print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
      print('\t - Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')

    # Compute mean metrics for the epoch
      epoch_val_metrics.append({
          'accuracy': sum(val_accuracy) / len(val_accuracy),
          'precision': sum(val_precision) / len(val_precision) if len(val_precision) > 0 else float('nan'),
          'recall': sum(val_recall) / len(val_recall) if len(val_recall) > 0 else float('nan'),
          'specificity': sum(val_specificity) / len(val_specificity) if len(val_specificity) > 0 else float('nan')
      })


  # Save the model using pickle
  with open("/content/drive/MyDrive/UFRGS/mestrado/bert_class_data_aug/models/metrics_model_topic_{}.pkl".format(topic), "wb") as f:
      pickle.dump(epoch_val_metrics, f)

  # Save the model using pickle
  with open("/content/drive/MyDrive/UFRGS/mestrado/bert_class_data_aug/models/model_topic_{}.pkl".format(topic), "wb") as f:
      pickle.dump(model, f)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch:   2%|▎         | 1/40 [00:06<04:11,  6.46s/it]


	 - Train loss: 0.6866
	 - Validation Accuracy: 0.7734
	 - Validation Precision: 0.6695
	 - Validation Recall: 0.8213
	 - Validation Specificity: 0.6828



Epoch:   5%|▌         | 2/40 [00:11<03:39,  5.78s/it]


	 - Train loss: 0.5411
	 - Validation Accuracy: 0.8125
	 - Validation Precision: 0.7260
	 - Validation Recall: 0.8236
	 - Validation Specificity: 0.7591



Epoch:   8%|▊         | 3/40 [00:17<03:27,  5.60s/it]

...


	 - Train loss: 0.0004
	 - Validation Accuracy: 0.8730
	 - Validation Precision: 0.8165
	 - Validation Recall: 0.9667
	 - Validation Specificity: 0.7689



Epoch:  98%|█████████▊| 39/40 [02:17<00:03,  3.52s/it]


	 - Train loss: 0.0004
	 - Validation Accuracy: 0.8730
	 - Validation Precision: 0.8165
	 - Validation Recall: 0.9667
	 - Validation Specificity: 0.7689



Epoch: 100%|██████████| 40/40 [02:20<00:00,  3.52s/it]


	 - Train loss: 0.0004
	 - Validation Accuracy: 0.8730
	 - Validation Precision: 0.8165
	 - Validation Recall: 0.9667
	 - Validation Specificity: 0.7689

